In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome() # 드라이버 초기화

# 로그인
login_url = 'https://member.melon.com/muid/web/login/login_informM.htm'
driver.get(login_url)

# 로그인 입력 위치
username_field = driver.find_element(by=By.ID, value='id')  # Replace 'username' with the actual ID of your username field
password_field = driver.find_element(by=By.ID, value='pwd')   # Replace 'password' with the actual ID of your password field

# 로그인
username_field.send_keys('tnwlstnrud123')
password_field.send_keys('3510**tn')
password_field.send_keys(Keys.RETURN)

driver.implicitly_wait(5)

In [2]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
from bs4 import BeautifulSoup

# Access capabilities
capabilities = driver.capabilities

# 차트 화면 열기
driver.get("https://www.melon.com/chart/month/index.htm?classCd=GN0000&moved=Y&rankMonth=202302")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# 필요 정보 가져오는 함수
def melon_collector(url):
    time.sleep(3)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 가져오기
    title=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank01') # 제목

    title2=[]
    for i in title:
        title2.append(i.text)

    del title2[100:]    
    
    # 가수 가져오기
    singer=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank02') # 가수

    singer2=[]
    for i in singer:
        singer2.append(i.text)
    del singer2[100:]
    
    # 가사 가져오기
    
    song_info = soup.find_all('div', attrs={'class': 'ellipsis rank01'})

    # Top 50만 추출
    songid = []
    time.sleep(2)
    for i in range(100):
        try:
            songid.append(re.sub(r'[^0-9]', '', song_info[i].find("a")["href"][42:])) # 곡 id정보 추출
        except:
            songid.append('')
            continue 

    dates = []    
    lyrics=[]
    genres=[]
    
    for i in songid:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
            time.sleep(2)
            date=driver.find_element(by=By.XPATH, value="/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[2]")
            dates.append(date.text)

            lyric = driver.find_element(by=By.CLASS_NAME, value="lyric")
            lyrics.append(lyric.text)
            
            genre= driver.find_element(by=By.XPATH, value="/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[3]")
            genres.append(genre.text)
        except: # 데이터가 없는경우
            lyrics.append("")
            continue
    dates2 = []
    lyrics2 = []
    genres2 = []

    for i in dates:
        dates2.append(i.replace("\n"," "))
    for i in lyrics:
        lyrics2.append(i.replace("\n"," "))
    for i in genres:
        genres2.append(i.replace("\n"," "))
        
    df=pd.DataFrame({"제목":title2,"가수":singer2,"가사":lyrics2, "장르": genres2, "발매일":dates2})
    
    # 저장하기
    df.to_csv(f'멜론{start}.csv', index = False)
    # 크롤링한 데이터를 csv형태로 저장하였고, 이 때 현재 멜론에서 들을 수 없는 노래나 가사가 등록되어있지 않은 노래들은 제외하고 가져와줍니다. 

start = 202301
url = 'https://www.melon.com/chart/month/index.htm?classCd=GN0000&moved=Y&rankMonth='

while start<=202310:
    new_url = url + str(start)
    melon_collector(new_url)
    print(start,'완료') # 완료 여부 표시
    start += 1

202301 완료
202302 완료
202303 완료
202304 완료
202305 완료
202306 완료
202307 완료
202308 완료
202309 완료
202310 완료
